## Model verification
To make sure than any of our modifications to the STEMMUS_SCOPE matlab code do not change the model results, this notebook is designed to check the new results to a previously calculated baseline.

In [3]:
import os
from pathlib import Path
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from PyStemmusScope import forcing_io
from PyStemmusScope import iostreamer
from PyStemmusScope import soil_io

### Enter the path to your local STEMMUS_SCOPE repository here:

In [4]:
# path to model repository
path_to_model = "path to the stemmus scope model directory"

### Update/set config files

In [5]:
# the user must provide the correct path
# path to config file
path_to_config_template_file = path_to_model + "/config_file_snellius.txt"
# Instantiate working directories handler from PyStemmusScope
config_template = iostreamer.read_config(path_to_config_template_file)

In [6]:
# edit config to run for 5 days
config_template["NumberOfTimeSteps"] = "240"

# Select a station. NOTE: the following stations do not work; DK-Lva, ES-ES1, ES-ES2, and IT-Noe
config_template["ForcingFileName"] = 'US-GLE_2009-2014_FLUXNET2015_Met.nc'

In [7]:
# Uncomment the following line to see all stations
#!ls '/projects/0/einf2480/forcing/plumber2_data/'

### Create input directories, prepare input files 

In [10]:
# Specify the forcing filename
forcing_filename = config_template["ForcingFileName"]
input_dir, output_dir, config_run_path = iostreamer.create_io_dir(forcing_filename, config_template)

config_run = iostreamer.read_config(config_run_path)
# Prepare the forcing data
forcing_io.prepare_forcing(input_dir, Path(config_run["ForcingPath"]) / forcing_filename, config_run)
# Generate the soil parameter input data for STEMMUS_SCOPE
soil_io.prepare_soil_data(config_run['SoilPropertyPath'], input_dir, config_run)

### Run the model

In [11]:
# generate a text file
path_to_code = path_to_model + "/src"

# set matlab log dir to slurm, otherwise java.log files are created in user home dir
os.environ['MATLAB_LOG_DIR'] = str(input_dir)

# set matlab arguments
path_to_config = f"'{config_run_path}'"
command_line = f'matlab -r "STEMMUS_SCOPE_exe({path_to_config});exit;"'
args = [command_line, "-nodisplay", "-nosplash", "-nodesktop"]

# run the model
result = subprocess.run(args, cwd = path_to_code, shell=True)
result.check_returncode()

MATLAB is selecting SOFTWARE OPENGL rendering.
Opening log file:  /scratch-shared/ecoextreml/stemmus_scope/input/US-GLE_2022-07-05-1400/java.log.55686

                            < M A T L A B (R) >
                  Copyright 1984-2021 The MathWorks, Inc.
             R2021a Update 3 (9.10.0.1684407) 64-bit (glnxa64)
                                May 27, 2021

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
Reading config from /scratch-shared/ecoextreml/stemmus_scope/input/US-GLE_2022-07-05-1400/US-GLE_2022-07-05-1400_config.txt

 The calculations start now 


### Generate the output netCDF file

In [13]:
# convert csv files to nc files
path_to_generate_netcdf_script = path_to_model + "/utils/csv_to_nc/generate_netcdf_files.py"
path_to_Variables_will_be_in_NetCDF_file = path_to_model + "/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv"
result = subprocess.run(["python", path_to_generate_netcdf_script,
     "--config_file", config_run_path, "--variable_file",
     path_to_Variables_will_be_in_NetCDF_file])
result.check_returncode()

/scratch-shared/ecoextreml/stemmus_scope/output/US-GLE_2022-07-05-1400/ECdata.csv
Reading variable metadata from /gpfs/home1/schilperb/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/output/US-GLE_2022-07-05-1400/US-GLE_2022-07-05-1400_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /scratch-shared/ecoextreml/stemmus_scope/output/US-GLE_2022-07-05-1400/US-GLE_2022-07-05-1400_STEMMUS_SCOPE.nc


### Load in the new netCDF as well as the verification netCDF file

In [14]:
station_name = forcing_filename[:6]
verification_directory = Path('/projects/0/einf2480/run_1year/v1.1.5') / 'output'
verification_nc_file = list(verification_directory.glob(f'{station_name}*/*STEMMUS_SCOPE.nc'))[0]

run_nc_file = list(Path(output_dir).glob(f'*STEMMUS_SCOPE.nc'))[0]

ds_ver = xr.open_dataset(verification_nc_file).squeeze(['x', 'y'])
ds_run = xr.open_dataset(run_nc_file).squeeze(['x', 'y'])

ds_ver = ds_ver.isel(time=slice(None, ds_run.time.size))

### Ensure that no significant changes to the model output have occured due to changes made elsewhere
Make sure this passes without errors before merging the new matlab code into the `python-compliant` branch. Preferably with more than a single station tested.

In [52]:
for key in ds_ver.keys():
#     if key in ['LWnet', 'LWup', 'Qle', 'Qh', 'Qg', 'VegT']:
#         pass
#     else:
    np.testing.assert_allclose(
        ds_ver[key],
        ds_run[key],
        rtol=1e-4,
        atol=np.abs(ds_ver[key]).mean().values / 100  # to allow for a small difference when the variable's value crosses 0.
    )

AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.577427

Mismatched elements: 128 / 240 (53.3%)
Max absolute difference: 5.8766937
Max relative difference: 1.5478772
 x: array([ -22.8349  ,  -23.30592 ,  -24.5375  ,  -19.94353 ,  -20.57276 ,
        -23.40292 ,  -23.87448 ,  -25.19518 ,  -20.80424 ,  -19.41654 ,
        -17.32158 ,  -10.80373 ,   -9.585564,   -9.994702,   -7.649747,...
 y: array([ -23.14748 ,  -23.62176 ,  -24.9116  ,  -20.25259 ,  -20.8946  ,
        -23.72201 ,  -24.20266 ,  -25.53323 ,  -21.08464 ,  -19.66676 ,
        -17.54344 ,  -10.94056 ,   -9.69978 ,  -10.11492 ,   -7.739421,...